## Parte 1

Neste exercício, tomando o código abaixo como ponto de partida, você vai desenvolver uma rede neural capaz de classificar diferentes artigos de vestuário. Para isso, você deverá implementar uma rede neural do tipo perceptron utilizando apenas NumPy (sem o auxílio de bibliotecas como TensorFlow, Theano, Torch, Caffe).

Para a realização deste exercício utilizaremos a base de dados [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist). A base de dados consiste em imagens monocromáticas de tamanho 28x28 pixels, cada imagem acompanhada de uma etiqueta que identifica a classe à qual ela pertence -- a classe representa a categoria de artigo de vestuário mostrado na imagem.

Critérios de avaliação:
- 1) Implementação da rede neural com pelo menos uma camada escondida e função ou método de *Forward propagation* (1,5 ponto)
- 2) Implementação do algoritmo de *back propagation*  (2,5 pontos)
- 3) Definição adequada da função de erro e cálculo da derivada no algoritmo de *back propagation*  (1,5 ponto)
- 4) Treinamento demonstrando redução gradual do erro nos dados de treinamento (1,5 ponto)
- 5) Treinamento demonstrando redução gradual do erro nos dados de validação  (1,5 ponto)
- 6) Mostrar alcance de pelo menos 84% de exatidão nos dados de validação  (1,5 ponto)
- **Ponto Extra:** Alcance de 87% ou mais de exatidão nos dados de validação  (1,0 ponto)

In [ ]:
# Importa as bibliotecas que serão utilizadas

import csv
import numpy as np
from matplotlib import pyplot as plt
import random
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Configura precisão para impressão de arrays do NumPy
np.set_printoptions(formatter={'float':lambda x: '%+01.2f ' % x})

In [ ]:
# Define dicionário para as etiquetas que
# representam cada classe

label_dict = {0:'T-shirt/top', 1:'Trouser', 2:'Pullover', 3:'Dress', 4:'Coat', 5:'Sandal',\
              6:'Shirt', 7:'Sneaker', 8:'Bag', 9:'Ankle boot' }

# Define função auxiliar para retornar o nome
# da classe correspondente para cada índice

def get_label(label_array):
    return label_dict[np.argmax(label_array)]

In [ ]:
# Define função auxiliar para leitura dos
# dados a partir de arquivos no formato CSV

def readfile(filename, total):
    with open(filename,'r') as csvfile:
        
        rows = csv.reader(csvfile)
        labels = np.zeros((total,10),dtype=np.float32)
        images = np.zeros((total,28*28),dtype=np.float32)
        i = 0
        
        for row in rows:
            if i > 0:
                data = np.array([float(x) for x in row]).astype(np.float32)
                labels[i-1,:] = (np.arange(0,10) == data[0]).astype(np.float32)
                images[i-1,:] = (data[1:] / 255.0) - 0.5
            i = i + 1
        
        return images, labels

In [ ]:
# Lê os dados de treinamento

train_images, train_labels = readfile('fashion-mnist_train.csv',60000)

In [ ]:
# Lê os dados de validação

test_images, test_labels = readfile('fashion-mnist_test.csv',10000)

In [ ]:
# Mostra alguns exemplos de dados para verificação
# (a cada execução mostra uma amostra diferente)

i = np.random.randint(0,test_images.shape[0])
image = test_images[i,:].reshape((28,28))
print(i)
print(get_label(test_labels[i,:]))
plt.imshow(image)
plt.show()

In [ ]:
#size_data            = len(train_images)      # numero de dados totais
#size_data_training   = test_labels.shape[0]   # numero de dados para treinamento
Size_colun_data       = train_images.shape[1]  # numero de colunas de dados

In [ ]:
# Rede neural # Caixa Preta

# Função Sigmóide - Fixa
def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

# Questão 1
class RedeNeural:
    
    def __init__(self,num_neuronios):
        # Configurações da rede
        self.size_data_validation = test_images.shape[0] # numero de dados para validação
        self.tot_colunas_saida    = test_labels.shape[1] # quantidade de valores possiveis na saida
    
        # Camada escondida/hidden
        # Gera matriz 784 x 100 com valores aleatórios entre -1 e 1
        self.Wh     = np.random.random((Size_colun_data,num_neuronios))*2.0 - 1.0
        # Gera matriz 1 x 100 com valores aleatórios entre -1 e 1
        self.bh     = np.random.random((1,num_neuronios))*2.0 - 1.0
        
        # Camada de saída/output
        # Gera matriz 100 x 10 com valores aleatórios entre -1 e 1
        self.Wo     = np.random.random((num_neuronios,self.tot_colunas_saida))*2.0 - 1.0
        # Gera matriz 1 x 10 com valores aleatórios entre -1 e 1
        self.bo     = np.random.random((1,self.tot_colunas_saida))*2.0 - 1.0
        
        # Delta camada escondida e saída
        # Gera matriz de 1 x 100 com valores entre -1 e 1
        self.deltah = np.random.random((1,num_neuronios))*2.0 - 1.0
        # Gera matriz de 1 x 10 com valores entre -1 e 1
        self.deltao = np.random.random((1,self.tot_colunas_saida))*2.0 - 1.0
    
    def forward(self, entrada):
        # s denota a entrada para a função de ativação
        self.sh = np.dot(entrada,self.Wh) + self.bh
        
        # z denota a saída após a função de ativação
        self.zh = sigmoid(self.sh)
        
        self.so = np.dot(self.zh,self.Wo) + self.bo
        self.zo = sigmoid(self.so)
        
        return self.zo
    
    # Questão 2
    def train(self, entradas, saidas, taxa_correcao):
        
        self.eta = taxa_correcao
        Err = 0.0
        
        #1 Apresenta a entrada x e calcula sh e zh para todas camadas
        for i in range(entradas.shape[0]):
            # entrada passa para 1x784 
            entrada = entradas[i,:].reshape(1,Size_colun_data)    # Entrada é a imagem
            # saida passa para 1x10
            saida = saidas[i,:].reshape(1,self.tot_colunas_saida) # Categoria da imagem # parece one-hot
            
            self.forward(entrada)
            
            err = self.zo - saida
            Err = Err + np.sum(err**2) / 2
            
            #3 Calcula delta saida para cada neuronio de saida
            self.deltao = (self.zo - saida)*self.zo*(1-self.zo)
          
            #4 Calcula delta camada escondida para cada neurônio das camadas escondidas até a entrada
            self.deltah = np.dot(self.deltao,self.Wo.T)*self.zh*(1-self.zh)
          
            # Questão 3
            #5 Atualizar os pesos(W) e biases(b)
            self.Wo = self.Wo - self.eta * np.dot(self.zh.T, self.deltao)
            self.bo = self.bo - self.eta * self.deltao
            
            self.Wh = self.Wh - self.eta * np.dot(entrada.T,self.deltah) 
            self.bh = self.bh - self.eta * self.deltah
        
        return Err

In [ ]:
def Teste(redes,labelsTest,entradasTest):
    numRedes = len(redes)
    numLabels = labelsTest.shape[0]
    numSaidas = labelsTest.shape[1]
    
    saida = np.zeros((numRedes,numLabels,numSaidas))
    
    numAcertos = 0
    numErros = 0
    
    for i in range(numRedes):
        saida[i,:,:] = redes[i].forward(entradasTest)
    
    for i in range(numLabels):
        escolhe = 0
        resposta = 0
        respostaCerta = 0
    
        for j in range(numSaidas):
            soma = saida[:,i,j].sum()
            
            if(labelsTest[i,j] == 1):
                respostaCerta = j
            
            if(escolhe < soma):
                escolhe = soma
                resposta = j
        
        if(resposta == respostaCerta):
            numAcertos = numAcertos + 1
        else:
            numErros = numErros + 1
    
    pxAcertos = (100/(numLabels)) * numAcertos
    return pxAcertos

In [ ]:
#instancia a rede neural
numRedes = 2
alfa = 0.5;
quant_neuronios = 100

redes = []
saida = []

for i in range(numRedes):
    rn = RedeNeural(quant_neuronios)
    redes.append(rn)
    saida.append(rn.forward(train_images))

In [ ]:
# faz o treino
numGeracao = 0;

for i in range(20):
    numGeracao = numGeracao +1
    print('\nGeracao: ',numGeracao)
    k = 0
    
    for rn in list(redes):
        k = k + 1;
        Err = rn.train(train_images,train_labels,alfa)
        rr = [rn]
        pxx = Teste(rr,train_labels,train_images)
        print('   Rede:',k,'  Err =',Err," ", pxx,"%")
        alfa = ((100-pxx)/100)/(5+(numGeracao/30))
    
    px = Teste(redes,test_labels,test_images)
    print('\n',px,'% de acertos no teste')
    